In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import os
from datetime import datetime
import time

def file_config(start_url, i=2):
    # Configuration du driver
    driver_path = r"/usr/local/bin/chromedriver"
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--headless')
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    driver.set_page_load_timeout(6000)
    attempts = i
    while attempts >0:
        try :
            driver.get(start_url)
            driver.maximize_window()
            attempts = 0
        except TimeoutException:
            attempts = attempts - 1
    return driver


driver = file_config(start_url = r'https://www.yapgi-immobilier.com/appartements.html')    

wait = WebDriverWait(driver, 30)
    
data = []

In [5]:
# Attendre que les éléments soient visibles
cards_presence = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'body > div.page-wrapper > section.blog-grid-section')))
# Créer une liste de cartes
cards = cards_presence.find_elements(By.CSS_SELECTOR, 'h3')

links = list()
    
for card in cards:
    link = card.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
    links.append(link)

for link in links:
    attempts = 3
    while attempts >0:
        try :
            driver.get(link)
            driver.refresh()
            time.sleep(10)
            attempts = 0
        except TimeoutException:
            attempts = attempts - 1
    
    # Utilisation de BeautifulSoup pour récupérer les détails de l'annonce
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try :
        details = soup.select_one('body > div.page-wrapper > div.sidebar-page-container > div > div')
    except AttributeError:
        continue    
    
    try:
        # from details_group1
        price = details.select_one("div.content-side.col-lg-9.col-md-8.col-sm-12.col-xs-12 > div > div > h2:nth-child(2)").text.strip()
        price = price.split(":")[-1]
        if 'habitation et' in price.lower():
            price = price.lower().split(" habitation et ")[0]
            
        title = details.select_one("div.content-side.col-lg-9.col-md-8.col-sm-12.col-xs-12 > div > div > h2:nth-child(1)").text.strip()
    except AttributeError:
        price = None
        title = None
        

    localisation = None
    immo_type = "Appartement"

    # from details_group2
    try:
        superficie = None
        nb_pieces = None
        nb_salle_de_bain = None
    except AttributeError:
        superficie = None
        nb_pieces = None
        nb_salle_de_bain = None

    # from details_group3
    try:
        description1 = details.select_one('div.content-side.col-lg-9.col-md-8.col-sm-12.col-xs-12 > div > div > div.text > blockquote > span').text.strip().replace('\n', ' ')
        description2 = details.select_one("div.sidebar-side.col-lg-3.col-md-4.col-sm-12.col-xs-12 > aside").text.strip().replace('\n', ' ')
        description = description1 + description2
    except AttributeError:
        description = None
    
    try:
        annonceur = "yapgi-immobilier"
    except AttributeError:
        annonceur = None
    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    data.append({
        'title': title,
        'price': price,
        'localisation': localisation,
        'superficie': superficie,
        "type d'immobilier": immo_type,
        'nb_pieces': nb_pieces,
        "nb_salle_de_bain": nb_salle_de_bain,
        'scraping_date': current_datetime,
        "annonceur" : annonceur,
        "link" : link,
        'description': description
    })
    
    print(data[-1])
    print(len(data))

driver.quit()


{'title': 'Appartemnet 4 Pièces au Plateau', 'price': '  1 000 000F CFA', 'localisation': None, 'superficie': None, "type d'immobilier": 'Appartement', 'nb_pieces': None, 'nb_salle_de_bain': None, 'scraping_date': '2024-07-24 12:12:40', 'annonceur': 'yapgi-immobilier', 'link': 'https://www.yapgi-immobilier.com/locat334.html', 'description': "Plateau Cite esculape, en face de la BCEAO, appartement de 4 pièces au 7eme étage avec une belle vue, cuisine, buanderie, séjour, terrasse couverte, une chambre avec salle d'eau, 2 chambres se partageant la même salle d'eau, WC visiteur.LOCAT334    GRAND SEJOUR   3 CHAMBRES    4 SALLES D'EAU  BALCON"}
1
{'title': "Appartemnet 3 pièces Riviera triangle en face de l'ancien camp d'akouedo", 'price': '  350 000F CFA', 'localisation': None, 'superficie': None, "type d'immobilier": 'Appartement', 'nb_pieces': None, 'nb_salle_de_bain': None, 'scraping_date': '2024-07-24 12:12:56', 'annonceur': 'yapgi-immobilier', 'link': 'https://www.yapgi-immobilier.com/

,title,price,localisation,superficie,type d'immobilier,nb_pieces,nb_salle_de_bain,scraping_date,annonceur,link,description
0,Appartemnet 4 Pièces au Plateau,1 000 000F CFA,None,None,Appartement,None,None,2024-07-24 12:12:40,yapgi-immobilier,https://www.yapgi-immobilier.com/locat334.html,"Plateau Cite esculape, en face de la BCEAO, ap..."
1,Appartemnet 3 pièces Riviera triangle en face ...,350 000F CFA,None,None,Appartement,None,None,2024-07-24 12:12:56,yapgi-immobilier,https://www.yapgi-immobilier.com/locat343.html,1 appartement de 3 pièces au 4e étage Dans i...
2,Appartemnet 3 Pièces à ANOUMABO,300 000F CFA,None,None,Appartement,None,None,2024-07-24 12:13:13,yapgi-immobilier,https://www.yapgi-immobilier.com/locat314.html,Immeuble neuf avec magasin et appartement. Mag...
3,Appartemnet 3 Pièces ANGRE 7ème TRANCHE,350 000F CFA,None,None,Appartement,None,None,2024-07-24 12:13:27,yapgi-immobilier,https://www.yapgi-immobilier.com/locat228.html,None
4,APPARTEMENTS 3 PIÈCES / M'POUTO cité verdoyante,350 000F CFA,None,None,Appartement,None,None,2024-07-24 12:13:41,yapgi-immobilier,https://www.yapgi-immobilier.com/locat332.html,2Chambres autonomes Séjour éclairé Split Cha...
5,Appartemnet 4 Pièces Cocody riviera Golf ambas...,1 500 000F CFA,None,None,Appartement,None,None,2024-07-24 12:13:56,yapgi-immobilier,https://www.yapgi-immobilier.com/locat138.html,Résidence avec piscine et groupe électrogène. ...
6,Appartemnet 4 Pièces à BASSAM,230 000F CFA,None,None,Appartement,None,None,2024-07-24 12:14:11,yapgi-immobilier,https://www.yapgi-immobilier.com/locat259.html,Bassam quartier bon berger appartement de 3 ch...
7,Appartemnet 3 pièces Riviera faya,350 000F CFA,None,None,Appartement,None,None,2024-07-24 12:14:30,yapgi-immobilier,https://www.yapgi-immobilier.com/locat340.html,Appartement de 3 pièces / Riviera Faya 3 eme ...
8,Vallon / appartements 4 pièces,500 000 / mois,None,None,Appartement,None,None,2024-07-24 12:14:43,yapgi-immobilier,https://www.yapgi-immobilier.com/locat338.html,Dans un immeuble de standing propre et sécuris...
9,Appartemnet 3pièces riviera 2,400 000F CFA,None,None,Appartement,None,None,2024-07-24 12:15:00,yapgi-immobilier,https://www.yapgi-immobilier.com/locat341.html,Appartement 3 pièces au 2e étage Parking in...


In [6]:
data = pd.DataFrame(data)
data

import locale
# Définir la locale en français
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

# Obtenir la date et l'heure actuelles
current_datetime = datetime.now()
formatted_date = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
formatted_month = current_datetime.strftime("%B")
formatted_day = current_datetime.strftime("%d")


import platform
if platform.system() == 'Windows':
    dynamic_path = f'D:\\Bureau\\MemoiresStages\\Travaux_techniques\\Scrapping\\Datasets\\{formatted_month}\\{formatted_day}_{formatted_month}\\Yapgi_appartements_{formatted_day}_{formatted_month}.csv'
else:
    dynamic_path = f'/mnt/d/Bureau/MemoiresStages/Travaux_techniques/Scrapping/Datasets/{formatted_month}/{formatted_day}_{formatted_month}/Yapgi_appartements_{formatted_day}_{formatted_month}.csv'

os.makedirs(os.path.dirname(dynamic_path), exist_ok=True)

data.to_csv(dynamic_path, index=False)